# 2017-05-28
---
I need find a way to bypass the *fl_image()* function limitations.
It's easy with MoviePy to retrieve the video frame rate :

    print(video.fps)
    print(processed_video.fps)
    ================
    25.0
    25.0
    
I'll try to build the script into a class, to save parameters between each frame iteration. Or use openCV again (but we try to use it as little as possible to avoid compiling as mush as possible).  
I need to integrate : recording of a video file by the Pi, sending that file -frame by frame- to server upon request, that may (or may not) do some processing.

# Outline
---
- [ ] Introduce comparaisons between frames to only code moving pixels
    - [X] Solve an issue : script correctly modifying only the first frame
    - [X] Solve an issue : "write file" command don't contain modifications
    - [ ] Solve an issue : badly processed frames
        - [ ] Find a way to substract frame with the previous one
- [X] Convert the video file to mp4 instead of avi
- [X] Capture a video with a stable background and moving object
- [ ] Implement the scripts into OpenRetina
- [ ] Make the Pi do the GC layer work 
- [ ] [OPT] Add a way to delete the video file on the Pi after sending and processing are done.
- [ ] [OPT] Try to find the optimal chunk size for a faster file transfer.
- [ ] [OPT] Rewrite the scripts to introduce classes
- [ ] [OPT] Dive into the video processing steps to find out when the sixth second (if the video is 5s long) is added

Going further :
- [ ] Study the program bottlenecks (Where does the program lose computing time?)
    - [ ] Add flags to monitors each script
- [ ] Switch from synchronous (phrs grabbing and sending frames when receiving other layers requests) to asynchronous (phrs grabbing continously and sending frames when requested) processes